### Simple Data Analyzer Agent
This project demonstrates how a GenAI-powered agent can analyze structured data and extract meaningful insights with minimal human intervention. To simulate real-world applications, I generated synthetic yet realistic datasets using AI, as acquiring real-world data for every use case can often be difficult or restricted.

The primary goal of this agent is not academic or research-based benchmarking but rather problem-solving through intelligent automation. It reflects a practical approach to how AI can be leveraged to understand, interpret, and act on data — especially in scenarios where real data is unavailable or sensitive.

In [31]:
import os
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents import AgentType
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI

In [32]:
# Load environment variables
from dotenv import load_dotenv
import os

# Load environment variables and set OpenAI API key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [33]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set seed for reproducibility
np.random.seed(42)

# Number of rows
n_rows = 1000

# Generate random dates
start_date = datetime(2023, 1, 1)
dates = [start_date + timedelta(days=np.random.randint(0, 365)) for _ in range(n_rows)]

# Simulate dataset values
data = {
    'Date': dates,
    'CustomerID': [f"CUST{1000+i}" for i in range(n_rows)],
    'Gender': np.random.choice(['Male', 'Female', 'Other'], n_rows),
    'Age': np.random.randint(18, 70, n_rows),
    'Region': np.random.choice(['North', 'South', 'East', 'West'], n_rows),
    'MembershipStatus': np.random.choice(['Silver', 'Gold', 'Platinum'], n_rows),
    'DeviceType': np.random.choice(['Mobile', 'Desktop', 'Tablet'], n_rows),
    'TimeOnSite': np.random.exponential(scale=10, size=n_rows).round(2),  # Time in minutes
    'PagesViewed': np.random.poisson(lam=5, size=n_rows),
    'CartValue': np.random.uniform(0, 500, n_rows).round(2),
    'Purchase': np.random.choice([0, 1], n_rows, p=[0.6, 0.4]),
    'SatisfactionScore': np.random.randint(1, 6, n_rows)
}

# Create DataFrame and sort
df = pd.DataFrame(data).sort_values('Date')




In [22]:
# Preview
print("\nSample Data:")
print(df.sample(5))



Sample Data:
          Date CustomerID Gender  Age Region MembershipStatus DeviceType  \
415 2023-01-05   CUST1415  Other   25  North         Platinum    Desktop   
16  2023-05-30   CUST1016   Male   32  South         Platinum     Mobile   
610 2023-10-10   CUST1610   Male   67   East         Platinum     Mobile   
14  2023-06-01   CUST1014   Male   57  North         Platinum    Desktop   
174 2023-08-13   CUST1174  Other   22  South             Gold    Desktop   

     TimeOnSite  PagesViewed  CartValue  Purchase  SatisfactionScore  
415       26.96            6     342.85         0                  5  
16        22.80            6     254.54         0                  3  
610       11.16            8     362.35         0                  5  
14         1.48            4     448.74         1                  1  
174       13.92            3     452.94         0                  4  


In [23]:
## duplicate values
df.duplicated().sum()

0

In [24]:
## Dataframe information
df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 848 to 889
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               1000 non-null   datetime64[ns]
 1   CustomerID         1000 non-null   object        
 2   Gender             1000 non-null   object        
 3   Age                1000 non-null   int64         
 4   Region             1000 non-null   object        
 5   MembershipStatus   1000 non-null   object        
 6   DeviceType         1000 non-null   object        
 7   TimeOnSite         1000 non-null   float64       
 8   PagesViewed        1000 non-null   int64         
 9   CartValue          1000 non-null   float64       
 10  Purchase           1000 non-null   int64         
 11  SatisfactionScore  1000 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(4), object(5)
memory usage: 101.6+ KB


In [25]:
print("\nSummary Statistics:")
print(df.describe(include='all'))


Summary Statistics:
                              Date CustomerID Gender          Age Region  \
count                         1000       1000   1000  1000.000000   1000   
unique                         NaN       1000      3          NaN      4   
top                            NaN   CUST1848   Male          NaN  North   
freq                           NaN          1    339          NaN    263   
mean    2023-07-01 13:22:04.800000        NaN    NaN    43.172000    NaN   
min            2023-01-01 00:00:00        NaN    NaN    18.000000    NaN   
25%            2023-04-08 00:00:00        NaN    NaN    30.000000    NaN   
50%            2023-07-01 12:00:00        NaN    NaN    42.500000    NaN   
75%            2023-09-27 06:00:00        NaN    NaN    56.000000    NaN   
max            2023-12-31 00:00:00        NaN    NaN    69.000000    NaN   
std                            NaN        NaN    NaN    14.994774    NaN   

       MembershipStatus DeviceType   TimeOnSite  PagesViewed    Ca

In [26]:
system_prompt = """
You are an intelligent AI assistant that helps extract insights from a pandas DataFrame.

Your capabilities include:
1. Describing and analyzing the structure of the dataset, including column names, data types, and missing values.
2. Performing descriptive statistical operations such as:
   - Mean, Median, Mode
   - Standard Deviation, Variance
   - Range, IQR, Min/Max
   - Quartiles
   - Skewness and Kurtosis
   - Univariate and Bivariate analysis
   - Normality tests and distribution checks
3. Performing feature engineering tasks, including:
   - Identifying and handling missing or null values
   - Detecting and handling outliers using techniques like IQR, Z-score
   - Filling missing values using mean, median, mode, or advanced imputation methods
   - Encoding categorical features using techniques such as:
     - One-hot encoding
     - Label encoding
     - Ordinal encoding
   - Scaling and normalizing numerical features
4. Performing correlation analysis:
   - Generating correlation matrices (e.g., Pearson, Spearman, Kendall)
   - Identifying strongly correlated features (positive or negative)

Guidelines:
- Always check column names and data types before performing operations.
- Explain your results briefly and clearly.
- Use concise, actionable language in your responses.
- When appropriate, suggest next steps based on analysis.

Respond based on the structure and content of the provided DataFrame. If a column or operation is not found or supported, clearly state that.
"""


In [34]:
from langchain.schema import SystemMessage

In [35]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        messages=[SystemMessage(content=system_prompt)]
    ),
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

/opt/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3493: UserWarning: WARNING! messages is not default parameter.
                messages was transferred to model_kwargs.
                Please confirm that messages is what you intended.
  if await self.run_code(code, result, async_=asy):


In [36]:
def ask_agent(question: str):
    """Function to ask questions to the agent and display the response."""
    try:
        print(f"\nQuestion: {question}")
        response = agent.run(question)
        print(f"Answer: {response}")
        print("---")
    except Exception as e:
        print(f"Error: {e}")


In [37]:
ask_agent("What is the average CartValue for each MembershipStatus?")
ask_agent("Are there any outliers in TimeOnSite?")
ask_agent("Show correlation matrix for all numeric features.")



Question: What is the average CartValue for each MembershipStatus?


> Entering new AgentExecutor chain...


/var/folders/b1/06nddx952f1g_2r63y2n569c0000gn/T/ipykernel_3706/3780525988.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(question)



Invoking: `python_repl_ast` with `{'query': "df.groupby('MembershipStatus')['CartValue'].mean()"}`


MembershipStatus
Gold        246.439968
Platinum    250.087898
Silver      247.572279
Name: CartValue, dtype: float64The average CartValue for each MembershipStatus is as follows:
- Gold: 246.44
- Platinum: 250.09
- Silver: 247.57

> Finished chain.
Answer: The average CartValue for each MembershipStatus is as follows:
- Gold: 246.44
- Platinum: 250.09
- Silver: 247.57
---

Question: Are there any outliers in TimeOnSite?


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\n\ndf = pd.DataFrame({\n    'Date': ['2023-01-01 00:00:00']*5,\n    'CustomerID': ['CUST1848', 'CUST1677', 'CUST1648', 'CUST1755', 'CUST1991'],\n    'Gender': ['Other', 'Male', 'Male', 'Female', 'Male'],\n    'Age': [60, 66, 31, 68, 28],\n    'Region': ['West', 'North', 'West', 'North', 'North'],\n    'MembershipStatus': ['Platinum', 'Platinum', 'Silver', 'Platinum